In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# specify GPU
device = torch.device("cuda")

In [ ]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
# from tqdm import tqdm
# import tqdm.notebook as tqdm
!pip install demoji
!pip install contractions
import demoji
import contractions

demoji.download_codes()
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 286kB 5.2MB/s 
     |████████████████████████████████| 327kB 33.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85395 sha256=c0c648cd61284f8c6cf73e3bb62de9b843e09878283a45231f003b05b483fdb9
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick
... OK (Got response in 0.53 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
!pip install transformers
!pip install mosestokenizer
!pip install sentencepiece
!pip install git+https://github.com/makcedward/nlpaug.git numpy matplotlib python-dotenv

     |████████████████████████████████| 2.5MB 4.1MB/s 
     |████████████████████████████████| 3.3MB 17.0MB/s 
     |████████████████████████████████| 901kB 20.9MB/s 
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-cp37-none-any.whl size=49120 sha256=1b87822a8e2ffbbaec67415b8bc38943d88a0357801e3e63570a8682ce3a4ef6
  Stored in directory: /root/.cache/pip/wheels/a2/e7/48/48d5e0f9c0cd5def2dfd7cb8543945f906448ed1313de24a29
  Created wheel for uctools: filename=uctools-1.3.0-cp37-none-any.whl size=6163 sha256=eeffcf6547060ed3ce5610f9f29db092f3b0d53c245a901deb9f70e5cb72b6a4
  Stored in directory: /root/.cache/pip/wheels/06/b6/8f/935d5bf5bca85d47c6f5ec31641879bba057d336ab36b1e773
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-cp37-none-any.whl size=3356 sha256=a24886ee0e4b04bc396bbfdbee4898a873cf731703e1d38dece57c8c7eab70ee
  Stored in directory: /root/.cache/pip/wheels/84/ea/29/e02f3b855bf19344972092873a1091b329309bbc3d3d0cbaef
Successfully built mosestokenizer

In [ ]:
import transformers
from tqdm.auto import tqdm
from transformers import AutoModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-cased')

In [ ]:
import nlpaug.augmenter.sentence as nas
import nlpaug.augmenter.word as naw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_json('/content/drive/MyDrive/sercasm/twitter/sarcasm_detection_shared_task_twitter_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sercasm/twitter/sarcasm_detection_shared_task_twitter_testing.jsonl',lines=True)

df['labels'] = df['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1['labels'] = df1['label'].apply(lambda x: ['SARCASM', 'NOT_SARCASM'].index(x))
df1.tail()

,label,context,response,id,labels
1795,NOT_SARCASM,[I have been a business customer of MWeb @USER...,@USER @USER @USER is definitely the best out t...,twitter_1796,1
1796,SARCASM,[A woman refuses to have her temperature taken...,@USER @USER Ye let her out run wild and infect...,twitter_1797,0
1797,SARCASM,[The reason big government wants @USER out is ...,"@USER @USER @USER Thanks for that , I would ha...",twitter_1798,0
1798,NOT_SARCASM,[Happy #musicmonday and #thanks for #all your ...,@USER @USER @USER Yes also #found this on #new...,twitter_1799,1
1799,NOT_SARCASM,[Not long wrapped on the amazing #January22nd ...,@USER @USER @USER you still need to send the l...,twitter_1800,1


In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@",'', text)
    text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
    text = re.sub(r"USER",'@USER', text)
    # text = re.sub(r"USER",'', text)
    text = re.sub(r'<URL>','',text)

    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    text = contractions.fix(text)
    return text

def preprocess_text2(text2):
    ans=[]
    for text in text2:
      text = re.sub(r"@",'', text)
      text = re.sub(r'\b(\w+)( \1\b)+', r'\1', text)
      text = re.sub(r"USER",'@USER', text)
      # text = re.sub(r"USER",'', text)
      text = re.sub(r'<URL>','',text)

      text = demoji.replace_with_desc(text)
      text = re.sub(r':','',text)
      text = contractions.fix(text)
      ans.append(text)
    return ans



In [ ]:
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
df['response']= df['response'].progress_apply(preprocess_text) 
df['context']=df['context'].progress_apply(preprocess_text2)

In [ ]:
def back_trans(text):
  back_translation_aug1 = naw.BackTranslationAug(
      device='cuda',
      from_model_name='Helsinki-NLP/opus-mt-ROMANCE-en',
      to_model_name='Helsinki-NLP/opus-mt-ROMANCE-en')
  back_translation_aug2 = naw.BackTranslationAug(
      device='cuda',
      from_model_name='facebook/wmt19-en-de',
      to_model_name='facebook/wmt19-de-en')
  return back_translation_aug2.augment(back_translation_aug1.augment(text))

print(back_trans('The quick brown fox jumped over the lazy dog'))

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


The speedy brown fox leapt over the lazy dog


In [ ]:
import nlpaug.augmenter.sentence as nas
aug = nas.ContextualWordEmbsForSentenceAug()
augmented_data = aug.augment('The quick brown fox jumped over the lazy dog')
print(augmented_data)


The quick brown fox jumped over the lazy dog V C The With 1 We The By X the I , , 0 U C It My The Image In The It The I , One What It B


In [ ]:
import nlpaug.augmenter.word as naw
aug = naw.AntonymAug()
# augmented_data = aug.augment(data)
augmented_data = aug.augment('The quick brown fox jumped over the lazy dog',n=1,num_thread=1)
print(augmented_data)


The quick brown fox jumped over the lazy dog


In [ ]:
import nlpaug.augmenter.word as naw
BackTranslationAug = naw.BackTranslationAug(device='cuda')
augmented_data = BackTranslationAug.augment('The quick brown fox jumped over the very lazy dog',n=1,num_thread=1)
print(augmented_data)



The fast brown fox jumped over the very lazy dog


In [ ]:
import nlpaug.augmenter.word as naw
ContextualWordEmbsAug = naw.ContextualWordEmbsAug(device='cuda')
augmented_data = ContextualWordEmbsAug.augment('The quick brown fox jumped over the very lazy dog',n=1,num_thread=1)
print(augmented_data)


the grey eyed fox went over the very lazy dog


In [ ]:
import nlpaug.augmenter.word as naw
aug = naw.RandomWordAug(action='substitute')
augmented_data = aug.augment('The quick brown fox jumped over the very lazy dog',n=1,num_thread=1)
print(augmented_data)
# action (str) – ‘substitute’, ‘swap’, ‘delete’ or ‘crop’

_ quick brown fox _ over _ very lazy dog


In [ ]:
# import nlpaug.augmenter.word as naw
# aug = naw.SpellingAug(dict_path='./spelling_en.txt')
# augmented_data = aug.augment('The quick brown fox jumped over the very lazy dog',n=1,num_thread=1)
# print(augmented_data)

In [ ]:
import nlpaug.augmenter.word as naw
SplitAug = naw.SplitAug()
augmented_data = SplitAug.augment('The quick brown fox jumped over the very lazy dog',n=1,num_thread=1)
print(augmented_data)

The q uick brown fox jumped ov er the very l azy dog


In [ ]:
import nlpaug.augmenter.word as naw
SynonymAug = naw.SynonymAug()
augmented_data = SynonymAug.augment('The quick brown fox jumped over the very lazy dog')
print(augmented_data)

The flying brown fox pass over over the very otiose dog


In [ ]:
# import nlpaug.augmenter.word as naw
# aug = naw.TfIdfAug(model_path='.')
# augmented_data = aug.augment('The quick brown fox jumped over the very lazy dog',n=1,num_thread=1)
# print(augmented_data)

In [ ]:
# from nlpaug.util.file.download import DownloadUtil
# DownloadUtil.download_glove('glove.6B', '/content/drive/MyDrive/sarcasm-detection/embeddings')


In [ ]:
import nlpaug.augmenter.word as naw
WordEbsAug = naw.WordEmbsAug(model_type='glove', model_path='/content/drive/MyDrive/sarcasm-detection/embeddings/glove.6B.50d.txt')
augmented_data = WordEbsAug.augment('The quick brown fox jumped over the lazy dog',n=1,num_thread=1)
print(augmented_data)

The follow reynolds fox jumped over the lazy dog


In [ ]:
def argument(text):
  text=BackTranslationAug.augment(text)
  # text=ContextualWordEmbsAug.augment(text)
  # text=SplitAug.augment(text)
  text=SynonymAug.augment(text)
  # text=WordEbsAug.augment(text)
  # text=back_trans(text)
  return text

def argument2(text):
  text=BackTranslationAug.augment(text)
  # text=ContextualWordEmbsAug.augment(text)
  # text=SplitAug.augment(text)
  # text=SynonymAug.augment(text)
  text=WordEbsAug.augment(text)
  # text=back_trans(text)
  return text

def argument3(text):
  text=BackTranslationAug.augment(text)
  text=ContextualWordEmbsAug.augment(text)
  # text=SplitAug.augment(text)
  # text=SynonymAug.augment(text)
  # text=WordEbsAug.augment(text)
  # text=back_trans(text)
  return text

def argument4(text):
  text=BackTranslationAug.augment(text)
  # text=ContextualWordEmbsAug.augment(text)
  # text=SplitAug.augment(text)
  # text=SynonymAug.augment(text)
  # text=WordEbsAug.augment(text)
  # text=back_trans(text)
  return text


In [ ]:
print(argument3(['The quick brown fox jumped over the lazy dog','The quick brown fox jumped over the lazy dog','The quick brown fox jumped over the lazy dog']))

['the fast brown one jumped over the lazy river', 'the fast brown fox jumped during your lazy dog', 'the fast brown lizard jumped over the young dog']


In [ ]:
df['response1']= df['response'].progress_apply(argument)

In [ ]:
df['response1'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/response1.csv')

In [ ]:
df['context1']=df['context'].progress_apply(argument)

In [ ]:
df['context1'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/context1.csv')

In [ ]:
df['response2']= df['response'].progress_apply(argument2) 

In [ ]:
df['response2'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/response2.csv')

In [ ]:
df['context2']=df['context'].progress_apply(argument2)

In [ ]:
df['context2'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/context2.csv')

In [ ]:
df['response3']= df['response'].progress_apply(argument3) 

In [ ]:
df['response3'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/response3.csv')

In [ ]:
df['context3']=df['context'].progress_apply(argument3)

In [ ]:
df['context3'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/context3.csv')

In [ ]:
df['response4']= df['response'].progress_apply(argument4) 

In [ ]:
df['response4'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/response4.csv')

In [ ]:
df['context4']=df['context'].progress_apply(argument4)

In [ ]:
df['context4'].to_csv('/content/drive/MyDrive/sarcasm-detection/argumented_data/context4.csv')